In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import requests

In [5]:
# Load our growth rates data
growth_data = pd.read_csv('../01_MergedData/data/top_counties.csv')

# Rename '2022' column to 'Income Growth'
growth_data = growth_data.rename(columns={'2022': 'Income Growth'})

In [6]:
growth_data.head()

,Unnamed: 0,GeoFips,GeoName,Income Growth,CAGR,Total Growth Rate
0,0,8031,"Denver, CO",1.342352,6.732323,18.007194
1,1,30031,"Gallatin, MT",1.126722,7.792158,39.259185
2,2,12021,"Collier, FL",1.081409,6.919575,23.419708
3,3,41017,"Deschutes, OR",1.056533,8.269753,30.814977
4,4,49049,"Utah, UT",1.048108,6.837103,35.070734


In [9]:
growth_data.head()

,Unnamed: 0,GeoFips,GeoName,Income Growth,CAGR,Total Growth Rate,Environmental Risk Score,Normalized Risk Score
0,0,8031,"Denver, CO",1.342352,6.732323,18.007194,0,NaN
1,1,30031,"Gallatin, MT",1.126722,7.792158,39.259185,0,NaN
2,2,12021,"Collier, FL",1.081409,6.919575,23.419708,0,NaN
3,3,41017,"Deschutes, OR",1.056533,8.269753,30.814977,0,NaN
4,4,49049,"Utah, UT",1.048108,6.837103,35.070734,0,NaN


In [16]:
# Function to fetch FEMA disaster declarations data
def fetch_fema_data(start_date, end_date):
    url = "https://www.fema.gov/api/open/v2/DisasterDeclarationsSummaries"
    params = {
        "$filter": f"declarationDate ge '{start_date}' and declarationDate le '{end_date}'",
        "$format": "json"
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return pd.DataFrame(response.json()['DisasterDeclarationsSummaries'])
    else:
        raise Exception(f"API request failed with status code {response.status_code}")

# Fetch FEMA data for the last 10 years
end_date = datetime.now().strftime('%Y-%m-%d')
start_date = (datetime.now() - timedelta(days=3650)).strftime('%Y-%m-%d')
fema_data = fetch_fema_data(start_date, end_date)

print(f"Fetched {len(fema_data)} FEMA disaster declarations")

# Function to calculate environmental risk score
def calculate_risk_score(county_data):
    if county_data.empty:
        return 0
    
    # Count the number of disasters
    disaster_count = len(county_data)
    
    # Calculate the average incident period (in days)
    county_data['incidentBeginDate'] = pd.to_datetime(county_data['incidentBeginDate'])
    county_data['incidentEndDate'] = pd.to_datetime(county_data['incidentEndDate'])
    avg_incident_period = (county_data['incidentEndDate'] - county_data['incidentBeginDate']).mean().days
    
    # Count unique disaster types
    unique_disaster_types = county_data['incidentType'].nunique()
    
    # Calculate risk score (this is a simple example, you might want to adjust the weights)
    risk_score = (disaster_count * 0.5) + (avg_incident_period * 0.3) + (unique_disaster_types * 0.2)
    
    return risk_score

# Clean county names
growth_data['CleanCounty'] = growth_data['GeoName'].apply(lambda x: x.split(',')[0].strip().lower())
growth_data['State'] = growth_data['GeoName'].apply(lambda x: x.split(',')[1].strip())

fema_data['CleanCounty'] = fema_data['designatedArea'].str.lower().str.replace(' county', '').str.strip()

# Calculate risk scores for each county in our growth data
risk_scores = []
for _, row in growth_data.iterrows():
    county_name = row['CleanCounty']
    state = row['State']
    
    county_data = fema_data[(fema_data['CleanCounty'] == county_name) & (fema_data['state'] == state)]
    
    if not county_data.empty:
        risk_score = calculate_risk_score(county_data)
        print(f"Calculated risk score for {county_name}, {state}: {risk_score}")
    else:
        risk_score = 0
        print(f"No disaster data found for {county_name}, {state}")
    
    risk_scores.append(risk_score)

# Add risk scores to our growth data
growth_data['Environmental Risk Score'] = risk_scores

# Normalize risk scores to a 0-100 scale
min_score = growth_data['Environmental Risk Score'].min()
max_score = growth_data['Environmental Risk Score'].max()
if min_score != max_score:
    growth_data['Normalized Risk Score'] = (growth_data['Environmental Risk Score'] - min_score) / (max_score - min_score) * 100
else:
    growth_data['Normalized Risk Score'] = 0  # All scores are the same

# Sort by Normalized Risk Score
growth_data = growth_data.sort_values('Normalized Risk Score', ascending=False)

# Display results
print("\nFinal Results:")
print(growth_data[['GeoName', 'Income Growth', 'CAGR', 'Total Growth Rate', 'Environmental Risk Score', 'Normalized Risk Score']])

# Save results to CSV
growth_data.to_csv('growth_rates_with_environmental_risk.csv', index=False)

print("\nAnalysis complete. Results saved to 'growth_rates_with_environmental_risk.csv'.")

Fetched 1000 FEMA disaster declarations
No disaster data found for denver, CO
No disaster data found for travis, TX
No disaster data found for flagler, FL
No disaster data found for davidson, TN
No disaster data found for charleston, SC
No disaster data found for missoula, MT
No disaster data found for canyon, ID
No disaster data found for salt lake, UT
No disaster data found for sarasota, FL
No disaster data found for hall, GA
No disaster data found for kootenai, ID
No disaster data found for pinal, AZ
No disaster data found for gallatin, MT
No disaster data found for washington, UT
No disaster data found for boulder, CO
No disaster data found for indian river, FL
No disaster data found for palm beach, FL
No disaster data found for benton, AR
No disaster data found for martin, FL
No disaster data found for utah, UT
No disaster data found for deschutes, OR
No disaster data found for collier, FL
No disaster data found for williamson, TN

Final Results:
             GeoName  Income Growt

In [15]:
growth_data.head(30) 

,Unnamed: 0,GeoFips,GeoName,Income Growth,CAGR,Total Growth Rate,Environmental Risk Score,Normalized Risk Score,CleanCounty,State
0,0,8031,"Denver, CO",1.342352,6.732323,18.007194,0,0,denver,CO
12,12,48453,"Travis, TX",0.900560,6.570435,28.679499,0,0,travis,TX
21,21,12035,"Flagler, FL",0.811683,6.634026,32.083438,0,0,flagler,FL
20,20,47037,"Davidson, TN",0.813328,7.190962,12.737907,0,0,davidson,TN
19,19,45019,"Charleston, SC",0.820833,7.252412,19.288897,0,0,charleston,SC
18,18,30063,"Missoula, MT",0.827218,6.713997,10.545783,0,0,missoula,MT
17,17,16027,"Canyon, ID",0.827298,9.421869,32.538484,0,0,canyon,ID
16,16,49035,"Salt Lake, UT",0.838304,6.912786,14.735035,0,0,salt lake,UT
15,15,12115,"Sarasota, FL",0.841621,7.417851,21.694562,0,0,sarasota,FL
14,14,13139,"Hall, GA",0.857762,6.725486,17.996372,0,0,hall,GA
